In [ ]:
#adaptive rag=query analysis + self rag

In [5]:
!pip -q install langchain_groq langchain_community tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.3 MB/s eta 0:

In [6]:
### Build Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# Set embeddings
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}

encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Docs to index
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=hf_embeddings,
)
retriever = vectorstore.as_retriever()

In [44]:
from dataclasses import field
##Router
from typing import Literal
from pydantic import BaseModel,Field
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

class query_router(BaseModel):
  datasource:Literal["web_search","vectorstore"]=Field(
      ...,
      description="Given a user's query choose to route the query  to vectorstore or web_search"
  )

llm=ChatGroq(model="llama3-8b-8192",temperature=0)
stuctured_llm=llm.with_structured_output(query_router)

system="""
You are an expert to route the user's query to vectorstore or web_search.
the Vectorstore contains data related to ai agengts, prompt engineering, and adversarial attacks.
use the vectorstore for questions on these topics. Otherwise, use web-search.
"""

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human","{query}")

    ]
)

choose_route_chain=prompt|stuctured_llm
query="What is AI agent?"
choose_route_chain.invoke({"query":query})



query_router(datasource='web_search')

In [10]:
choose_route_chain.invoke({"query":"who is Elon Musk?"})

query_router(datasource='web_search')

In [45]:
#retriever node
query="What is AI agent?"
retrieved_docs=retriever.get_relevant_documents(query)
retrieved_data=retrieved_docs[1].page_content

In [ ]:
retriever.invoke(query)

In [47]:
# generate node
from langchain_core.output_parsers import StrOutputParser
template="""You are an Ai Assistant ,you job is to generate answer to user's query based on the context.
here is the context:{context}
here is the query:{query}
answer:"""
rag_prompt=ChatPromptTemplate.from_template(template)
rag_chain=rag_prompt|llm|StrOutputParser()
query="What is AI agent?"
generated_data=rag_chain.invoke({"context":retrieved_data,"query":query})
generated_data

"Based on the provided context, I'd be happy to help!\n\nAccording to the context, an AI agent is a type of autonomous agent that uses a Large Language Model (LLM) as its main controller. This AI agent is designed to make decisions independently without seeking user assistance, and it has a short-term memory limit of approximately 4000 words. The agent's primary goal is to optimize believability at the moment versus in time, taking into account relationships between agents and observations of one agent by another."

In [48]:
retrieved_data

'Planning is essentially in order to optimize believability at the moment vs in time.\nPrompt template: {Intro of an agent X}. Here is X\'s plan today in broad strokes: 1)\nRelationships between agents and observations of one agent by another are all taken into consideration for planning and reacting.\nEnvironment information is present in a tree structure.\n\n\n\n\nFig. 13. The generative agent architecture. (Image source: Park et al. 2023)\nThis fun simulation results in emergent social behavior, such as information diffusion, relationship memory (e.g. two agents continuing the conversation topic) and coordination of social events (e.g. host a party and invite many others).\nProof-of-Concept Examples#\nAutoGPT has drawn a lot of attention into the possibility of setting up autonomous agents with LLM as the main controller. It has quite a lot of reliability issues given the natural language interface, but nevertheless a cool proof-of-concept demo. A lot of code in AutoGPT is about for

In [49]:
# relevance or not checking grade

class grade_documents(BaseModel):
  binary_score:str=Field(
      description=("retrieved documents are relevant,'yes',or,'no'")

  )
relevance_llm=llm.with_structured_output(grade_documents)
system="""
You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
"""
relevance_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human","retrieved_documents:\n\n{documents}\nquery:\n\n{query}")
    ]
)
relevance_grader=relevance_prompt|relevance_llm
query="What is AI agent?"
retrieved_docs=retriever.get_relevant_documents(query)
retrieved_data=retrieved_docs[1].page_content
print(retrieved_data)
score=relevance_grader.invoke({"documents":retrieved_data,"query":query})
score.binary_score

Planning is essentially in order to optimize believability at the moment vs in time.
Prompt template: {Intro of an agent X}. Here is X's plan today in broad strokes: 1)
Relationships between agents and observations of one agent by another are all taken into consideration for planning and reacting.
Environment information is present in a tree structure.




Fig. 13. The generative agent architecture. (Image source: Park et al. 2023)
This fun simulation results in emergent social behavior, such as information diffusion, relationship memory (e.g. two agents continuing the conversation topic) and coordination of social events (e.g. host a party and invite many others).
Proof-of-Concept Examples#
AutoGPT has drawn a lot of attention into the possibility of setting up autonomous agents with LLM as the main controller. It has quite a lot of reliability issues given the natural language interface, but nevertheless a cool proof-of-concept demo. A lot of code in AutoGPT is about format parsing.


'yes'

In [76]:
# hallucinate grader

class hallucinate_checker(BaseModel):
  binary_score:str=Field(
      description=("Answer is grounded to facts ,'yes',or,'no'")
  )

hallucinate_llm=llm.with_structured_output(hallucinate_checker)
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucinate_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generated_data}")
    ]
)

hallucinate_grader=hallucinate_prompt|hallucinate_llm

score=hallucinate_grader.invoke({"documents":retrieved_data,"generated_data":generated_data})
score.binary_score

'yes'

In [51]:
# question rewriter
# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for vectorstore retrieval. Look at the input and try to reason about the underlying sematic intent / meaning. Just provide the new question alone no need for any other content."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Here is the initial question: \n\n {question} \n Formulate an improved question."),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
query="What is AI agent?"
question_rewriter.invoke({"question": query})

'What are the characteristics and functionalities of an artificial intelligence agent?'

In [84]:
# Answer Grader


class GradeAnswer(BaseModel):


    binary_score: str = Field(description="Answer addresses the question, 'yes' or 'no'")



structured_llm_grader = llm.with_structured_output(GradeAnswer)


system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generated_data}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
score=answer_grader.invoke({"question": query,"generated_data": generated_data})
score.binary_score

'no'

In [53]:
#web search
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)

In [54]:
# graph state
from typing_extensions import TypedDict
from typing import List

class GraphState(TypedDict):
    query:str
    generated_data:str
    documents:List[str]



In [35]:
!pip -q install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.3 MB/s eta 0:00:00


In [85]:
#graph flow define
from langgraph.graph import END, StateGraph
from langchain.schema import Document

#nodes
def retrieve(state):
  print("---RETRIEVE---")
  query=state['query']

  documents=retriever.invoke(query)
  return {"documents":documents,"query":query}



def generate(state):
  print("---GENERATE---")
  query=state['query']
  retrieved_data=state['documents']
  generated_data=rag_chain.invoke({"context":retrieved_data,"query":query})

  return {"documents":retrieved_data,"query":query,"generated_data":generated_data}

def gradeing_docs(state):
  print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
  query=state['query']
  documents=state['documents']
  filtered_docs=[]
  for doc in documents:
    score=relevance_grader.invoke({"documents":doc.page_content,"query":query})
    grade=score.binary_score
    if grade=='yes':
      print("---GRADE: DOCUMENT RELEVANT---")
      filtered_docs.append(doc)
    else:
      print("---GRADE: DOCUMENT NOT RELEVANT---")
      continue
  return {"documents": filtered_docs, "query": query}

def transform_query(state):


    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}




def web_search(state):


    print("---WEB SEARCH---")
    query = state["query"]

    # Web search
    docs = web_search_tool.invoke({"query": query})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "query": query}


def searching_route(state):
  print("---ROUTE QUESTION---")
  query=state['query']
  # docs = web_search_tool.invoke({"query": question})
  res=choose_route_chain.invoke({"query":query})
  if res.datasource=='web_search':
    print("---ROUTE QUESTION TO WEB SEARCH---")
    return "web_search"
  else:
    print("---ROUTE QUESTION TO RAG---")
    return "vectorstore"


def decide_to_generate(state):
  print("---ASSESS GRADED DOCUMENTS---")
  query = state["query"]
  filtered_documents = state["documents"]
  if not filtered_documents:
    print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---")
    return "transform_query"
  else:
    print("---DECISION: GENERATE---")
    return "generate"

def hallucinate_check(state):
  print("---CHECK HALLUCINATIONS---")
  query=state["query"]
  generated_data=state['generated_data']
  documents = state["documents"]

  score=hallucinate_grader.invoke({"documents":documents,"generated_data":generated_data})
  grade=score.binary_score
  if grade=='yes':
    print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
    print("Grade--checking ans is addresses the query---")
    score=answer_grader.invoke({"question": query,"generated_data": generated_data})
    grade=score.binary_score
    if grade=="yes":
      print("---DECISION: GENERATION ADDRESSES QUESTION---")

      return "useful"
    else:
      print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
      return "not useful"
  else:
    print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
    return "not supported"





In [86]:
workflow=StateGraph(GraphState)

#adding nodes
workflow.add_node("web_search", web_search)
workflow.add_node("retrieve",retrieve)
workflow.add_node("gradeing_docs", gradeing_docs)
workflow.add_node("generate", generate)
workflow.add_node("transform_query", transform_query)

#edges

workflow.set_conditional_entry_point(
    searching_route,
    {
        "web_search":"web_search",
        "vectorstore":"retrieve"
    }
)
workflow.add_edge("web_search","generate")
workflow.add_edge("retrieve","gradeing_docs")
workflow.add_conditional_edges(
    "gradeing_docs",
    decide_to_generate,
    {
        "transform_query":"transform_query",
        "generate":"generate"
    }
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    hallucinate_check,
    {
        "useful":END,
        "not useful":"transform_query",
        "not supported":"generate"
    }

)

In [87]:
app=workflow.compile()

In [89]:
from pprint import pprint
input={"query":"what is ai agent memory?","documents":[]}
for output in app.stream(input):
  for key,value in output.items():
    print(f"node---- : {key}")
  print("\n--\n")
print(value['generated_data'])


---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
node---- : retrieve

--

---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
node---- : gradeing_docs

--

---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
Grade--checking ans is addresses the query---
---DECISION: GENERATION ADDRESSES QUESTION---
node---- : generate

--

According to the provided context, AI agent memory refers to the ability of an LLM-powered autonomous agent to retain and recall information over extended periods. This is achieved through the use of an external vector store and fast retrieval mechanism, which enables the agent to access and utilize its past experiences and knowledge.

In the context of the LLM-powered autonomous agent system, there are two types of memory:

1. **Short-term

In [91]:
input={"query":"what is ai agent?","documents":[]}
for output in app.stream(input):
  for key,value in output.items():
    print(f"node---- : {key}")
  print("\n--\n")
print(value['generated_data'])

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
node---- : web_search

--

---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
Grade--checking ans is addresses the query---
---DECISION: GENERATION ADDRESSES QUESTION---
node---- : generate

--

An artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals.
